In [21]:
import math as m
import numpy as np
from scipy.linalg import eigh
import matplotlib.pyplot as plt
%matplotlib inline



In [22]:
def S(basis) :
    N = basis.shape[0]
    Overlap_matrix = np.zeros((N,N))
    for p in range(N) :
        for q in range(p,N) :
            Overlap_matrix[p,q] = (np.pi/(basis[p] + basis[q]))**(3/2)
            Overlap_matrix[q,p] = Overlap_matrix[p,q]
    return Overlap_matrix

def T(basis) :
    N = basis.shape[0]
    Kintetic_matrix = np.zeros((N,N))
    for p in range(N) :
        for q in range(p,N) :
            Kintetic_matrix[p,q] = 3*basis[p]*basis[q]*np.pi**(3/2)/(basis[p] + basis[q])**(5/2)
            Kintetic_matrix[q,p] = Kintetic_matrix[p,q]
    return Kintetic_matrix

def A(basis) :
    N = basis.shape[0]
    Coulomb_matrix = np.zeros((N,N))
    for p in range(N) :
        for q in range(p,N) :
            Coulomb_matrix[p,q] = -4*np.pi/(basis[p] + basis[q])
            Coulomb_matrix[q,p] = Coulomb_matrix[p,q]
    return Coulomb_matrix


In [23]:
def Q(basis) : 
    N = basis.shape[0]
    Electron_matrix = np.zeros((N,N,N,N))
    for p in range(N) :
        for q in range(p+1) :
            for r in range(p+1) :
                if p != q :
                    for s in range(r+1) :
                        Electron_matrix[p,r,q,s] = 2*np.pi**(5/2)/(basis[p] + basis[q])/(basis[r] + basis[s])/(basis[p] + basis[q] + basis[r] + basis[s])**(1/2)
                        Electron_matrix[p,s,q,r] = Electron_matrix[p,r,q,s]
                        Electron_matrix[q,r,p,s] = Electron_matrix[p,r,q,s]
                        Electron_matrix[q,s,p,r] = Electron_matrix[p,r,q,s]
                        Electron_matrix[r,p,s,q] = Electron_matrix[p,r,q,s]
                        Electron_matrix[r,q,s,p] = Electron_matrix[p,r,q,s]
                        Electron_matrix[s,p,r,q] = Electron_matrix[p,r,q,s]
                        Electron_matrix[s,q,r,p] = Electron_matrix[p,r,q,s]
                else :
                    for s in range(q+1) :
                        Electron_matrix[p,r,q,s] = 2*np.pi**(5/2)/(basis[p] + basis[q])/(basis[r] + basis[s])/(basis[p] + basis[q] + basis[r] + basis[s])**(1/2)
                        Electron_matrix[p,s,q,r] = Electron_matrix[p,r,q,s]
                        Electron_matrix[q,r,p,s] = Electron_matrix[p,r,q,s]
                        Electron_matrix[q,s,p,r] = Electron_matrix[p,r,q,s]
                        Electron_matrix[r,p,s,q] = Electron_matrix[p,r,q,s]
                        Electron_matrix[r,q,s,p] = Electron_matrix[p,r,q,s]
                        Electron_matrix[s,p,r,q] = Electron_matrix[p,r,q,s]
                        Electron_matrix[s,q,r,p] = Electron_matrix[p,r,q,s]
    return Electron_matrix

In [36]:
def normalize(C,S) :
    I = 0
    N = np.shape(C)[0]
    for r in range(N) :
        for s in range(N) : 
            I = I + S[r,s]*C[r]*C[s]
    C = C/np.sqrt(I)
    return C

1.4142135623730951

In [33]:
def Helium(a0,a1,a2,a3,C) :
    basis = np.array([a0, a1, a2, a3])
    
    S_matrix = S(basis)
    T_matrix = T(basis)
    A_matrix = A(basis)
    Q_tensor = Q(basis)
    Q_matrix = np.zeros((4,4)) 

    C = normalize(C,S_matrix)
    
    for r in range(4) : 
        for s in range(4) : 
            Q_matrix = Q_matrix + Q_tensor[:,r,:,s]*C[r]*C[s]
    
    eigenenergy, eigenvector= eigh((T_matrix + A_matrix + Q_matrix), S_matrix)
    eigenenergy = np.min(eigenenergy)
    eigenvector = eigenvector[:,np.argmin(eigenenergy)]
    
    total_energy = 0 
    for p in range(4) :
        for q in range(4) :
            total_energy = total_energy  + 2*(A_matrix[p,q]+T_matrix[p,q])*C[p]*C[q]
            for r in range(4) :
                for s in range(4) :
                    total_energy = total_energy + Q_tensor[p,r,q,s]*C[p]*C[r]*C[q]*C[s]
                    
    return eigenenergy, eigenvector, total_energy


In [35]:
def SCF(a0,a1,a2,a3) : 
    n = 10
    C = np.array([1,1,1,1])
    for ele in range(n) :
        total_energy = Helium(a0,a1,a2,a3,C)[2]
        C = Helium(a0,a1,a2,a3,C)[1]
        print(total_energy)
    return total_energy

SCF(0.298073, 1.242567, 5.782948, 38.474970)


-0.16233578267128507
-2.74905418936348
-2.8475909854591923
-2.8545950821335384
-2.855118239270491
-2.8551572371945677
-2.8551601477017536
-2.855160364859865
-2.855160381063693
-2.8551603822727585


-2.8551603822727585

In [27]:
def MGD(a0,a1,a2,a3,step_size,limit) :
    gradient = np.array([[SCF(a0+0.00001,a1,a2,a3) - SCF(a0,a1,a2,a3)], [SCF(a0,a1+0.00001,a2,a3) - SCF(a0,a1,a2,a3)], [SCF(a0,a1,a2+0.00001,a3) - SCF(a0,a1,a2,a3)], [SCF(a0,a1,a2,a3+0.00001) - SCF(a0,a1,a2,a3)]])/0.00001
    v = np.zeros((4,1))
    i = 0
    data = np.array([[i],[SCF(a0,a1,a2,a3)],[a0],[a1],[a2],[a3],[np.linalg.norm(gradient)]])
    while i < limit and np.linalg.norm(gradient) > 0.000001:
        v = step_size*gradient + 0.9*v
        a0 = a0 - v[0,0]
        a1 = a1 - v[1,0]
        a2 = a2 - v[2,0]
        a3 = a3 - v[3,0]
        i += 1
        gradient = np.array([[SCF(a0+0.00001,a1,a2,a3) - SCF(a0,a1,a2,a3)], [SCF(a0,a1+0.00001,a2,a3) - SCF(a0,a1,a2,a3)], [SCF(a0,a1,a2+0.00001,a3) - SCF(a0,a1,a2,a3)], [SCF(a0,a1,a2,a3+0.00001) - SCF(a0,a1,a2,a3)]])/0.000001
        data = np.append(data,[[i],[SCF(a0,a1,a2,a3)],[a0],[a1],[a2],[a3],[np.linalg.norm(gradient)]], axis = 1)
    return data

In [28]:
test = MGD(0.247,0.891,3.49,18.52,0.1,100)
for ele in range(1,7) :
    plt.figure(ele)
    plt.plot(test[0,:],test[ele,:])
print(test[:,-1])

-0.10585314150345856
-2.743467430194579
-2.8429815454649146
-2.850678045110751
-2.851307304353236
-2.8513586161889957
-2.851362806959898
-2.851363149099839
-2.8513631770358034
-2.8513631793167202
-0.10584826304856289
-2.74346653893893
-2.8429813285648318
-2.8506780238931917
-2.8513073112818814
-2.851358626408856
-2.851362817530042
-2.8513631597052296
-2.851363187644613
-2.8513631899258574
-0.10584890822352179
-2.74346622409455
-2.842981286169243
-2.8506780226489448
-2.8513073150380794
-2.8513586307092416
-2.851362821885977
-2.85136316406661
-2.8513631920065095
-2.851363194287801
-0.10584826304856289
-2.74346653893893
-2.8429813285648318
-2.8506780238931917
-2.8513073112818814
-2.851358626408856
-2.851362817530042
-2.8513631597052296
-2.851363187644613
-2.8513631899258574
-0.10584829176620292
-2.7434665886113065
-2.842981335809415
-2.850678024833083
-2.8513073114656327
-2.851358626511004
-2.8513628176238917
-2.85136315979828
-2.8513631877375807
-2.8513631900188146
-0.10584826304856289
-

KeyboardInterrupt: 